In [ ]:
import os
import shutil
import sys
import pathlib
import dill
import pickle as pkl
import json
import argparse
import random as rd

from contextlib import nullcontext

import numpy as np
import pandas as pd

import torch
import torchvision

from torch.utils.data import DataLoader, Subset

from fir.datasets import deep_fashion_ctsrbm
from fir.arch import backbones_cnn, backbones_trf, models, heads

import fir.utils.train
import fir.utils.log
import fir.utils.dict
import fir.utils.list
import fir.utils.mem
import fir.utils.sig
import fir.utils.time

from tqdm import tqdm

from time import time
from datetime import datetime

from itertools import chain
from functools import reduce

import json
import socket

from torch.distributed import init_process_group, destroy_process_group
from torch.utils.data.distributed import DistributedSampler
from torch.nn.parallel import DistributedDataParallel as DDP

import pprint

In [ ]:
backbone = backbones_cnn.ResNet50Backbone()

ctsrbm_dataset_dir = os.path.join(pathlib.Path.home(), "data", "DeepFashion", "Consumer-to-shop Clothes Retrieval Benchmark")
backbone_image_transform = backbone.get_image_transform()

ctsrbm_dataset = deep_fashion_ctsrbm.ConsToShopClothRetrBmkDataset(ctsrbm_dataset_dir, img_transform=backbone_image_transform, neg_img_filename_list_id="test")

ctsrbm_train_idxs = ctsrbm_dataset.get_subset_indices(split="train")

ctsrbm_train_dataset = Subset(ctsrbm_dataset, ctsrbm_train_idxs)

ctsrbm_train_loader = DataLoader(
    ctsrbm_train_dataset,
    batch_size=16,
    num_workers=4,
    pin_memory=True,
    shuffle=False
)

print("DataLoader size:", len(ctsrbm_train_loader))